<b>XGBoost</b> - it is one of the most popular gradient boosting algorithm. This is a tree based algorithm

In [4]:
import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn import preprocessing

In [5]:
df_ = pd.read_csv("input/cat_train_folds.csv")
df_.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,kfold
0,436027,0.0,0.0,0.0,T,Y,Blue,Trapezoid,Lion,Russia,...,1.0,Contributor,Boiling Hot,n,S,lS,3.0,8.0,1,0
1,498631,0.0,0.0,0.0,F,Y,Red,Trapezoid,Dog,Costa Rica,...,3.0,Novice,Warm,o,X,iS,2.0,11.0,0,0
2,372327,0.0,0.0,0.0,F,N,Blue,Triangle,Dog,India,...,3.0,Master,Hot,o,N,OZ,2.0,3.0,0,0
3,570356,0.0,0.0,1.0,T,N,Blue,Polygon,Hamster,Russia,...,3.0,Contributor,Lava Hot,h,Y,aA,4.0,3.0,0,0
4,368020,0.0,0.0,0.0,F,Y,Green,Polygon,Hamster,Finland,...,1.0,Novice,Warm,i,M,zc,7.0,11.0,0,0


In [7]:
def run(fold):
# load the full training data with folds
    df = pd.read_csv("input/cat_train_folds.csv")
    # all columns are features except id, target and kfold columns
    features = [
    f for f in df.columns if f not in ("id", "target", "kfold")
    ]
# fill all NaN values with NONE
# note that I am converting all columns to "strings"
# it doesnt matter because all are categories
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna("NONE")
    # now it’s time to label encode the features
    for col in features:
    # initialize LabelEncoder for each feature column
        lbl = preprocessing.LabelEncoder()
    # fit label encoder on all data
        lbl.fit(df[col])
    # transform all the data
        df.loc[:, col] = lbl.transform(df[col])
    # get training data using folds
    df_train = df[df.kfold != fold].reset_index(drop=True)
    # get validation data using folds
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    # get training data
    x_train = df_train[features].values
    # get validation data
    x_valid = df_valid[features].values
    # initialize xgboost model
    model = xgb.XGBClassifier(
        n_jobs=-1,
    max_depth=7,
    n_estimators=200
    )
    # fit model on training data (ohe)
    model.fit(x_train, df_train.target.values)
    # predict on validation data
    # we need the probability values as we are calculating AUC
    # we will use the probability of 1s
    valid_preds = model.predict_proba(x_valid)[:, 1]
    # get roc auc score
    auc = metrics.roc_auc_score(df_valid.target.values, valid_preds)
# print auc
    print(f"Fold = {fold}, AUC = {auc}")
if __name__ == "__main__":
    for fold_ in range(5):
        run(fold_)


Fold = 0, AUC = 0.7625249545749646
Fold = 1, AUC = 0.7641523204228844
Fold = 2, AUC = 0.7608105651174362
Fold = 3, AUC = 0.7610264972457682
Fold = 4, AUC = 0.7651760407878181
